### testing fill missing

In [1]:
import pandas as pd
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import FunctionTransformer
from pyspark.sql.functions import pandas_udf, PandasUDFType
import pathlib
import time
import pyspark.pandas as ps

import numpy as np

from datetime import date, datetime, timedelta
from pyspark.pandas.indexes.datetimes import DatetimeIndex
import pyspark
from pyspark.sql import SparkSession, Window
from pyspark.sql.functions import when, col, rank, lit, monotonically_increasing_id, date_trunc, udf, min, max, explode
from pyspark.sql.types import StructType, StructField, TimestampType, IntegerType, FloatType, DateType, StringType

In [2]:
conf = pyspark.SparkConf().setAll([\
    ('spark.app.name', 'ReduceData')])
spark = SparkSession.builder.config(conf=conf)\
    .getOrCreate()

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


23/05/10 02:34:43 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [3]:
glucose_data_schema=StructType([StructField('NumId', IntegerType(), True),
                                        StructField('PatientId', StringType(), True),
                                        StructField('Value', FloatType(), True),
                                        StructField('GlucoseDisplayTime', TimestampType(), True),
                                        StructField('GlucoseDisplayTimeRaw', StringType(), True),
                                        StructField('GlucoseDisplayDate', DateType(), True)])

In [4]:
startTime = time.time()
data_location = "/cephfs/train_test_val/train_set"
allPaths = [str(x) for x in list(pathlib.Path(data_location).glob('*.parquet')) if 'part-00' in str(x)][0]
#allPaths.sort()
# print(allPaths)

pyspark_glucose_data = spark.read \
                       .schema(glucose_data_schema) \
                       .format('parquet') \
                       .load(allPaths)
pyspark_glucose_data = pyspark_glucose_data.withColumn("GlucoseDisplayTime",
                                                       date_trunc("minute",
                                                       col("GlucoseDisplayTime")))

pyspark_glucose_data=pyspark_glucose_data.orderBy("PatientId",
                                                  "GlucoseDisplayTime",
                                                  ascending=True)

print(time.time()-startTime)

2.508166551589966


In [10]:
test_df = pyspark_glucose_data.toPandas()

/home/jovyan/glucose-data-analysis/glucose_venv/lib/python3.10/site-packages/pyspark/sql/pandas/conversion.py:248: FutureWarning: Passing unit-less datetime64 dtype to .astype is deprecated and will raise in a future version. Pass 'datetime64[ns]' instead
  series = series.astype(t, copy=False)


In [ ]:
len(test_df)

In [ ]:
min_max = test_df.groupBy('NumId')\
    .agg(min("GlucoseDisplayTime").alias('MinDate'),\
        max("GlucoseDisplayTime").alias('MaxDate'))

In [ ]:
min_max = min_max.withColumn("Index", monotonically_increasing_id())\
.select('Index','NumId','MinDate','MaxDate')

In [ ]:
min_max.show(4)

In [ ]:
min_max = min_max.toPandas()

In [ ]:
main_df = test_df.drop(columns=['GlucoseDisplayDate', 'PatientId', 'GlucoseDisplayTimeRaw'])

In [7]:
merged_schema = StructType([StructField('NumId', IntegerType(), True),
                            StructField('GlucoseDisplayTime', TimestampType(), True),
                           StructField('Value', IntegerType(), True)])

In [48]:
min_max = test_df.groupby('NumId')\
                .agg({'GlucoseDisplayTime' : ['min', 'max']})

merge_df = pd.DataFrame(columns=['GlucoseDisplayTime', 'NumId'])
for idx, row in min_max.iterrows():
    #grab all poteitnal dates in range
    
    date_df = pd.DataFrame(pd.date_range(row[0], row[1], freq='5min'), columns=['GlucoseDisplayTime'])                              
    date_df['NumId']= idx

    # merge dates with big pypsark df
    merged = test_df[test_df['NumId'] == idx]\
            .merge(date_df, how='right', on=['GlucoseDisplayTime', 'NumId'])\
            .sort_values(by=['GlucoseDisplayTime', 'Value'], na_position='last')

    merged['TimeLag'] = np.concatenate((merged['GlucoseDisplayTime'].iloc[1:].values,\
                                        np.array(merged['GlucoseDisplayTime'].iloc[-1])), axis=None)\
                        .astype('datetime64[ns]')

    merged['Diff'] = (merged['TimeLag'] - merged['GlucoseDisplayTime']).dt.seconds

    len_merged = len(merged)

    # get all index of rows with diff less than 5 mins, add 1 to remove next row, 
    # dont include last row to delete
    indexes_to_remove = [x for x in merged[merged['Diff'] < 300].index + 1 if x < len_merged]

    if len(indexes_to_remove) > 0:
        merged = merged.drop(indexes_to_remove)

    # merged is the patient's combined and deduplicated rows 
    # ready freddy for interpoletty

    break;

105045
[]


In [8]:
@pandas_udf(merged_schema, PandasUDFType.GROUPED_MAP)
def interpolation(self, df):    
    min_max = test_df.groupby('NumId')\
                .agg({'GlucoseDisplayTime' : ['min', 'max']})

    merge_df = pd.DataFrame(columns=['GlucoseDisplayTime', 'NumId'])
    for idx, row in min_max.iterrows():
        #grab all poteitnal dates in range

        date_df = pd.DataFrame(pd.date_range(row[0], row[1], freq='5min'), columns=['GlucoseDisplayTime'])                              
        date_df['NumId']= idx

        # merge dates with big pypsark df
        merged = test_df[test_df['NumId'] == idx]\
                .merge(date_df, how='right', on=['GlucoseDisplayTime', 'NumId'])\
                .sort_values(by=['GlucoseDisplayTime', 'Value'], na_position='last')

        merged['TimeLag'] = np.concatenate((merged['GlucoseDisplayTime'].iloc[1:].values,\
                                            np.array(merged['GlucoseDisplayTime'].iloc[-1])), axis=None)\
                            .astype('datetime64[ns]')

        merged['Diff'] = (merged['TimeLag'] - merged['GlucoseDisplayTime']).dt.seconds

        len_merged = len(merged)

        # get all index of rows with diff less than 5 mins, add 1 to remove next row, 
        # dont include last row to delete
        indexes_to_remove = [x for x in merged[merged['Diff'] < 300].index + 1 if x < len_merged]

        if len(indexes_to_remove) > 0:
            merged = merged.drop(indexes_to_remove)

        # its ready freddy for some interpoletty
        # grab all potential dates in range

        date_df = pd.DataFrame(pd.date_range(row[0], row[1], freq='5min'), columns=['GlucoseDisplayTime'])                              
        date_df['NumId']= idx

        # merge dates with big pypsark df
        merged = test_df[test_df['NumId'] == idx]\
                .merge(date_df, how='right', on=['GlucoseDisplayTime', 'NumId'])\
                .sort_values(by=['GlucoseDisplayTime', 'Value'], na_position='last')

        merged['TimeLag'] = np.concatenate((merged['GlucoseDisplayTime'].iloc[1:].values,\
                                            np.array(merged['GlucoseDisplayTime'].iloc[-1])), axis=None)\
                            .astype('datetime64[ns]')

        merged['Diff'] = (merged['TimeLag'] - merged['GlucoseDisplayTime']).dt.seconds

        len_merged = len(merged)

        # get all index of rows with diff less than 5 mins, add 1 to remove next row, 
        # dont include last row to delete
        indexes_to_remove = [x for x in merged[merged['Diff'] < 300].index + 1 if x < len_merged]

        if len(indexes_to_remove) > 0:
            merged = merged.drop(indexes_to_remove)

        # its ready freddy for some interpoletty
        # merged DF is the dataframe ready to go into interpolation function



In [11]:
interpolation(test_df)

TypeError: Invalid argument, not a string or column:          NumId                                     PatientId  Value  \
0         5868  0m18JauIIZlt6psiqeBS1SfN+URQ4axXv6xfTPIP2+M=  164.0   
1         5868  0m18JauIIZlt6psiqeBS1SfN+URQ4axXv6xfTPIP2+M=  164.0   
2         5868  0m18JauIIZlt6psiqeBS1SfN+URQ4axXv6xfTPIP2+M=  161.0   
3         5868  0m18JauIIZlt6psiqeBS1SfN+URQ4axXv6xfTPIP2+M=  158.0   
4         5868  0m18JauIIZlt6psiqeBS1SfN+URQ4axXv6xfTPIP2+M=  160.0   
...        ...                                           ...    ...   
1997486   4326  zN6N3TFBSOTHIVFbATwVRX+ecqxvHz6ZWZ67CPIpQmo=  125.0   
1997487   4326  zN6N3TFBSOTHIVFbATwVRX+ecqxvHz6ZWZ67CPIpQmo=  124.0   
1997488   4326  zN6N3TFBSOTHIVFbATwVRX+ecqxvHz6ZWZ67CPIpQmo=  119.0   
1997489   4326  zN6N3TFBSOTHIVFbATwVRX+ecqxvHz6ZWZ67CPIpQmo=  119.0   
1997490   4326  zN6N3TFBSOTHIVFbATwVRX+ecqxvHz6ZWZ67CPIpQmo=  117.0   

         GlucoseDisplayTime          GlucoseDisplayTimeRaw GlucoseDisplayDate  
0       2022-0

In [ ]:
tolerance = .01
fancy = merge_df.groupby(merge_df['GlucoseDisplayTime'].mul(tolerance).round()).max().reset_index(drop=True)

In [ ]:
@pandas_udf(merged_schema, PandasUDFType.GROUPED_MAP)
def test(df):
    # min_date = df.GlucoseDisplayTime.min()
    # max_date = df.GlucoseDisplayTime.max()
    merge_df = pd.DataFrame(columns=['GlucoseDisplayTime', 'NumId'])
    for idx in range(df):
        row = min_max[min_max.Index == idx].iloc[0]
        date_df = pd.DataFrame(pd.date_range(row.MinDate, row.MaxDate, freq='5min'), columns=['GlucoseDisplayTime'])                              
        date_df['NumId']= row.NumId
        print('asdfasdf')
        merge_df = pd.concat([merge_df, date_df], axis=0)
        
    return merge_df

In [ ]:
grouped = min_max.groupby('NumId').apply(test)

In [ ]:
grouped.show()

In [ ]:
def date_range_def(minDate, maxDate):    
    return ps.date_range(minDate, maxDate, freq='5min')

date_range_udf = udf(lambda x,y: date_range_def(x,y))

In [ ]:
min_max = min_max.withColumn('Range', date_range_udf(col('MinDate'), col('MaxDate')))

In [ ]:
min_max.show(5)

In [ ]:
dates = spark.sql("SELECT sequence(CAST('2022-04-08 07:49:00' AS DATE), CAST('2022-10-01 07:47:00' AS DATE), interval 5 minutes) as date").withColumn("date", explode(col("date")))

In [ ]:
dates.show()

In [ ]:
@pandas_udf(PandasUDFType.GROUPED_MAP)
def replace_missing(subset):
    """ INPUT
        subset:     spark DataFrame with 1 patient
        OUTPUT
        filled_df:  spark DataFrame with 1 patient (-1 columns) and all missing rows filled in; not sorted
    """

    '''get first and last date (takes about 10 seconds per ten days of one patient)'''
    minimum = subset.agg({'GlucoseDisplayTime': 'min'}).collect()[0][0]
    maximum = subset.agg({'GlucoseDisplayTime': 'max'}).collect()[0][0]

    '''make a range that fills all those in'''
    def date_range_list(start_date, end_date):
        if start_date > end_date:
            raise ValueError("start_date must come before end_date")

        datetime_list = []
        curr_date = start_date
        while curr_date <= end_date:
            datetime_list.append([curr_date])
            curr_date += timedelta(minutes=5)
        return datetime_list

    datetime_list = date_range_list(minimum, maximum)

    '''make a dataframe of those dates'''
    deptSchema = StructType([       
        StructField('GlucoseDisplayTime', TimestampType(), True)
    ])
    dt_df = self.spark.createDataFrame(data=datetime_list, schema=deptSchema)

    '''merge og dataframe back into the new one'''
    merged = subset.unionByName(dt_df, allowMissingColumns=True)

    '''get rid of the timestamps we already have (using the exact same method as from "drop duplicate datetimes for each patient" above)'''
    window = Window.partitionBy('GlucoseDisplayTime').orderBy('tiebreak')
    merged = (merged
     .withColumn('tiebreak', monotonically_increasing_id())
     .withColumn('rank', rank().over(window))
     .filter(col('rank') == 1).drop('rank','tiebreak')
    )

    '''filling out the columns as needed:
        -PatientId should be all the same string
        -GlucoseDisplayTimeRaw should be used for checking the dates here, but implementation will have to come later'''
    merged = merged.fillna(patient_str, subset='PatientId')
    merged = merged.drop('GlucoseDisplayTimeRaw') #someday i'll have time to use this as the double-checker
    merged = merged.withColumn('GlucoseDisplayDate',
                               to_date(col('GlucoseDisplayTime')))

    """ ============== FILL IN MISSING VALUES ============== """
    # filler = subset.agg({'Value': 'median'}).collect()[0][0]
    filler = subset.agg({'Value': 'avg'}).collect()[0][0]

    filled_df = merged.fillna(filler, subset='Value')

    return filled_df
    

In [ ]:
test = test_df.groupBy('NumId').apply(lambda x: replace_missing(x))